In [2]:
from pydub import AudioSegment
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
import random
from sklearn.metrics import confusion_matrix
import pandas as pd
import torch
import torchaudio
from torch.utils.data import DataLoader, Dataset
from torch import nn
import torch.optim as optim
import torch.nn.functional as F
from pydub import AudioSegment
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [3]:
import os
result = os.environ['PATH']
os.environ['PATH'] += os.pathsep + 'C:\\Program Files\\ffmpeg-2024-09-22-git-a577d313b2-full_build\\bin'

In [4]:
final_audio = AudioSegment.from_file("final_audio_with_drinking_events.wav")
event_df = pd.read_csv("drinking_event_times.csv")

In [4]:
# rows = []

# for _, row in event_df.iterrows():
#     for onefourth_minute in range(row['start_onefourth_minute'], row['end_onefourth_minute'] + 1):
#         rows.append({"onefourth_minute": onefourth_minute, "water_drinking": 1})

# # Create a DataFrame from rows and remove duplicates
# drinking_df = pd.DataFrame(rows).drop_duplicates()

# # Fill in missing quarter-minute intervals with water_drinking = 0
# all_onefourth_minutes = range(drinking_df['onefourth_minute'].min(), drinking_df['onefourth_minute'].max() + 1)
# all_onefourth_minutes_df = pd.DataFrame({"onefourth_minute": all_onefourth_minutes})
# final_df = all_onefourth_minutes_df.merge(drinking_df, on="onefourth_minute", how="left")
# final_df['water_drinking'] = final_df['water_drinking'].fillna(0).astype(int)

# def split_and_label_audio(audio, final_df, sr=16000, window_size=30, overlap=15):
#     labeled_segments = []
#     step = window_size - overlap  # Compute step size based on overlap
    
#     for idx, row in final_df.iterrows():
#         # Convert 'onefourth_minute' to milliseconds
#         start_time = row['onefourth_minute'] * 15 * 1000  # Convert quarter minute to milliseconds
#         end_time = start_time + window_size * 1000         # 30 seconds later
        
#         # Check if the drinking event is in the current row or the next row
#         if row['water_drinking'] == 1:
#             label = 1
#         elif idx + 1 < len(final_df) and final_df.iloc[idx + 1]['water_drinking'] == 1:
#             label = 1
#         else:
#             label = 0
        
#         # Extract the audio segment based on calculated start and end times
#         segment = audio[start_time:end_time]
        
#         # Append the segment and its label
#         labeled_segments.append((segment, label))
        
#         # Move to the next window with overlap
#         start_time += step * 1000  # Move by step size in milliseconds
#         end_time = start_time + window_size * 1000  # Update end time

#     return labeled_segments

In [5]:
def extract_mfcc(audio_segment, sr=16000, n_mfcc=13):
    samples = np.array(audio_segment.get_array_of_samples()).astype(np.float32) / 32768.0
    mfcc = librosa.feature.mfcc(y=samples, sr=sr, n_mfcc=n_mfcc)
    return mfcc.T  # Transpose to (time, n_mfcc)


In [6]:
event_df

,start,end,sample_name
0,1041,10016,library_water_bottle_0_elvis_162_aug2
1,196518,206521,home_water_bottle_0_elvis_111_aug1
2,408802,419868,library_water_bottle_0_elvis_152_aug1
3,439012,450013,library_water_bottle_0_elvis_206_aug1
4,535584,546568,home_water_bottle_0_elvis_179_aug2
...,...,...,...
325,34464952,34475896,home_water_bottle_0_elvis_127_aug2
326,34513573,34521615,library_water_bottle_0_elvis_201_aug1
327,34728409,34738423,home_water_bottle_0_elvis_119_aug2
328,34803992,34815992,library_water_bottle_0_elvis_155_aug2


In [43]:
start = 0
end = 30

event_df[
    ((event_df['start_seconds'] >= start) & (event_df['start_seconds'] <= end)) |
    ((event_df['end_seconds'] >= start) & (event_df['end_seconds'] <= end))
]

,start,end,sample_name,start_seconds,end_seconds
0,1041,10016,library_water_bottle_0_elvis_162_aug2,1,10


In [7]:
# event_df start and end seconds
event_df['start_seconds'] = event_df['start'] // 1000
event_df['end_seconds'] = event_df['end'] // 1000

In [45]:
# create a new column called duration in seconds
event_df['duration'] = event_df['end_seconds'] - event_df['start_seconds']

In [46]:
# get sum 
event_df['duration'].sum()

3228

In [9]:
def split_audio(audio, event_df, sr=16000, window_size=30, overlap=15):
    segments = []
    step = window_size - overlap  # Compute step size based on overlap
    
    start_time = 0
    end_time = window_size * 1000  # Convert window size to milliseconds
    
    while end_time <= len(audio):
        # Extract the audio segment based on calculated start and end times
        segment = audio[start_time:end_time]
        
        # Check if the segment contains any event
        start_seconds = start_time // 1000
        end_seconds = end_time // 1000
        event_df[
            ((event_df['start_seconds'] >= start_seconds) & (event_df['start_seconds'] <= end_seconds)) |
            ((event_df['end_seconds'] >= start_seconds) & (event_df['end_seconds'] <= end_seconds))
        ]
        
        if len(event_df) > 0:
            label = 1
        else:
            label = 0
        
        # Append the segment
        segments.append(segment)
        
        # Move to the next window with overlap
        start_time += step * 1000  # Move by step size in milliseconds
        end_time = start_time + window_size * 1000  # Update end time

    return segments

In [11]:
# split the audio into 30 second segments
audio_segments = split_audio(final_audio, event_df)

In [12]:
class DeepAudioClassifierCNN(nn.Module):
    def __init__(self):
        super(DeepAudioClassifierCNN, self).__init__()
        # First Convolutional Block
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout1 = nn.Dropout2d(0.3)  # Regularization
        
        # Second Convolutional Block with Residual Connection
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(64)
        self.conv4 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(64)
        self.conv1x1_1 = nn.Conv2d(32, 64, kernel_size=1)  # Match residual channels
        self.dropout2 = nn.Dropout2d(0.3)
        
        # Third Convolutional Block with Residual Connection
        self.conv5 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.bn5 = nn.BatchNorm2d(128)
        self.conv6 = nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1)
        self.bn6 = nn.BatchNorm2d(128)
        self.conv1x1_2 = nn.Conv2d(64, 128, kernel_size=1)  # Match residual channels
        self.dropout3 = nn.Dropout2d(0.3)

        # Fully Connected Layers
        self.fc1 = nn.Linear(self._get_conv_output_size(), 256)
        self.fc2 = nn.Linear(256, 64)
        self.fc3 = nn.Linear(64, 2)  # Binary classification

    def _get_conv_output_size(self):
        # Create a dummy input to calculate the size after convolution layers
        dummy_input = torch.zeros(1, 1, 1876, 13)  # Adjust according to your MFCC input
        x = self.pool(F.relu(self.bn1(self.conv1(dummy_input))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))  # Apply until last conv layer
        x = self.pool(F.relu(self.bn5(self.conv5(x))))
        return int(torch.prod(torch.tensor(x.size()[1:])))
    
    def forward(self, x):
        # First Convolutional Block
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool(x)
        x = self.dropout1(x)

        # Second Convolutional Block with Residual Connection
        residual = self.conv1x1_1(x)  # Adjust residual channels to match conv output
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x)))
        x = x + residual  # Replace in-place addition
        x = self.pool(x)
        x = self.dropout2(x)

        # Third Convolutional Block with Residual Connection
        residual = self.conv1x1_2(x)  # Adjust residual channels to match conv output
        x = F.relu(self.bn5(self.conv5(x)))
        x = F.relu(self.bn6(self.conv6(x)))
        x = x + residual  # Replace in-place addition
        x = self.pool(x)
        x = self.dropout3(x)

        # Flatten for Fully Connected Layers
        x = x.view(x.size(0), -1)

        # Fully Connected Layers
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)  # No activation for output, use CrossEntropyLoss

        return x

# Create the model instance
model = DeepAudioClassifierCNN()

In [13]:
# load pytorch model
path = "audio_classifier_cnn.pth"
model.load_state_dict(torch.load(path))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

DeepAudioClassifierCNN(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout1): Dropout2d(p=0.3, inplace=False)
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv1x1_1): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1))
  (dropout2): Dropout2d(p=0.3, inplace=False)
  (conv5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(

In [14]:
# pass the audio segments through the model
predictions = []
for segment in tqdm(audio_segments):
    mfcc = extract_mfcc(segment)
    mfcc = torch.tensor(mfcc).unsqueeze(0).unsqueeze(0).to(device)
    output = model(mfcc)
    _, predicted = torch.max(output, 1)
    predictions.append(predicted.item())

100%|██████████| 2339/2339 [03:39<00:00, 10.68it/s]


In [15]:
# create a df of every second of the audio
all_seconds = len(final_audio) // 1000
all_seconds_df = pd.DataFrame({"second": range(all_seconds)})
all_seconds_df['prediction'] = 0

# fill in the predictions
for idx, row in event_df.iterrows():
    all_seconds_df.loc[
        (all_seconds_df['second'] >= row['start_seconds']) & (all_seconds_df['second'] <= row['end_seconds']),
        'prediction'
    ] = 1

In [ ]:
predictions  

In [17]:
# get the audios with their starting point as key where the prediction is 1
audio_segments_with_predictions = {}
for i, audio in enumerate(audio_segments):
    if predictions[i] == 1:
        start = i * 15
        audio_segments_with_predictions[start] = audio

In [19]:
len(audio_segments_with_predictions)

781

In [34]:
# split every 30 seconds of the audio into 1 second segments
one_second_segments = {}
for start, audio in audio_segments_with_predictions.items():
    for i in range(0, len(audio), 1000):
        segment = audio[i:i+1000]
        time = start + i // 1000
        one_second_segments[time] = segment

In [35]:
len(one_second_segments)

15645

In [31]:
# drop rows which are not in the one_second_segments
all_seconds_df['keep'] = False
for audio_start in audio_segments_with_predictions.keys():
    for i in range(0, 30):
        all_seconds_df.loc[all_seconds_df['second'] == audio_start + i, 'keep'] = True

In [32]:
all_seconds_df = all_seconds_df[all_seconds_df['keep']]

In [28]:
# reset index
all_seconds_df.reset_index(drop=True, inplace=True)

In [33]:
all_seconds_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15645 entries, 0 to 35009
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   second      15645 non-null  int64
 1   prediction  15645 non-null  int64
 2   keep        15645 non-null  bool 
dtypes: bool(1), int64(2)
memory usage: 382.0 KB


In [36]:
all_seconds_df['prediction'].value_counts() 

prediction
0    12113
1     3532
Name: count, dtype: int64

In [38]:
all_seconds_df.head()

,second,prediction,keep
0,0,0,True
1,1,1,True
2,2,1,True
3,3,1,True
4,4,1,True


In [ ]:
one_second_segments

In [79]:
def split_and_label_audio(audio_dict, event_df, sr=16000, window_size=2, overlap=1):
    """
    Create 3-second audio segments with 1-second overlap, labeling based on drinking events.
    
    Args:
        audio_dict (dict): Dictionary mapping seconds to audio segments.
        event_df (DataFrame): DataFrame with 'second' and 'prediction' columns.
        sr (int): Sample rate (default is 16000).
        window_size (int): Length of each window in seconds.
        overlap (int): Overlap between consecutive windows in seconds.
    
    Returns:
        list: List of tuples (AudioSegment, label).
    """
    labeled_segments = []
    step = window_size - overlap  # Compute step size for overlapping windows
    
    # Determine the range of seconds available in the audio dictionary
    audio_seconds = sorted(audio_dict.keys())
    min_second = min(audio_seconds)
    max_second = max(audio_seconds)
    
    # Iterate over the start times for the windows
    start_second = min_second
    while start_second + window_size <= max_second + 1:
        # Collect the audio for the current 3-second window
        segment_audio = []
        for second in range(start_second, start_second + window_size):
            if second in audio_dict:  # Check if the second has audio
                segment_audio.append(audio_dict[second])
        
        # Combine the audio segments into a single segment if audio exists
        if segment_audio:
            combined_audio = sum(segment_audio)  # Combine pydub AudioSegments
            
            # Determine the label for the window
            label = event_df[
                (event_df['second'] >= start_second) & (event_df['second'] < start_second + window_size)
            ]['prediction'].max()  # Label is 1 if any second in the window has prediction = 1
            
            labeled_segments.append((combined_audio, label))
        
        # Move the window forward
        start_second += step
    
    return labeled_segments

In [80]:
audio_segments = split_and_label_audio(audio_dict=one_second_segments, event_df=all_seconds_df)

# all_mfccs = []
# all_labels = []

# for segment, label in tqdm(audio_segments):
#     mfcc = extract_mfcc(segment)
#     all_mfccs.append(mfcc)
#     all_labels.append(label)

# # Train-test split
# X_train, X_test, y_train, y_test = train_test_split(all_mfccs, all_labels, test_size=0.25, random_state=42, stratify=all_labels)

In [81]:
mel_spectrogram = torchaudio.transforms.MelSpectrogram(
    sample_rate=16000,
    n_mels=64,
    n_fft=1024,
    hop_length=512
)

import torch.nn.functional as F

def preprocess_audio(audio_segments, sample_rate=16000):
    spectrograms = []
    max_width = 0  # Find the maximum width of the spectrograms
    
    for audio in audio_segments:
        waveform = torch.tensor(audio.get_array_of_samples()).float() / 32768.0
        waveform = waveform.unsqueeze(0)  # Add channel dimension
        spectrogram = torchaudio.transforms.MelSpectrogram(sample_rate)(waveform)
        spectrogram = torchaudio.transforms.AmplitudeToDB()(spectrogram)
        spectrograms.append(spectrogram)
        max_width = max(max_width, spectrogram.shape[-1])
    
    # Pad spectrograms to the same width
    padded_spectrograms = []
    for spectrogram in spectrograms:
        padding = (0, max_width - spectrogram.shape[-1])  # Pad only along the time axis
        padded_spectrogram = F.pad(spectrogram, padding, "constant", 0)
        padded_spectrograms.append(padded_spectrogram)
    
    return torch.stack(padded_spectrograms)

In [82]:
from torch.utils.data import Dataset, DataLoader
import torch

class AudioDataset(Dataset):
    def __init__(self, spectrograms, labels):
        self.spectrograms = spectrograms
        self.labels = labels
    
    def __len__(self):
        return len(self.spectrograms)
    
    def __getitem__(self, idx):
        return self.spectrograms[idx], self.labels[idx]
    
spectrograms = preprocess_audio([seg[0] for seg in audio_segments])  # List of audio segments
labels = torch.tensor([seg[1] for seg in audio_segments])           # Corresponding labels

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(spectrograms, labels, test_size=0.25, random_state=42, stratify=labels)

train_dataset = AudioDataset(X_train, y_train)
test_dataset = AudioDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


c:\Users\Jayesh\AppData\Local\Programs\Python\Python39\lib\site-packages\torchaudio\functional\functional.py:584: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  warnings.warn(


In [83]:
import torch
import torch.nn as nn
import torchaudio

# class AudioClassifier(nn.Module):
#     def __init__(self, num_classes=2):
#         super(AudioClassifier, self).__init__()
        
#         # CNN layers
#         self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
#         self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
#         self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
#         self.dropout = nn.Dropout(0.3)
#         self.relu = nn.ReLU()
        
#         # LSTM layers
#         self.lstm = nn.LSTM(
#             input_size=128,  # Match features dimension
#             hidden_size=256,  # Configurable
#             num_layers=2,  # Configurable
#             batch_first=True,  # Ensure batch is the first dimension
#             bidirectional=True  # Enable if bidirectional LSTM is needed
#         )
        
#         # Fully connected layers
#         self.fc1 = nn.Linear(128 * 2, 128)  # *2 because bidirectional
#         self.fc2 = nn.Linear(128, num_classes)
        
class AudioClassifier(nn.Module):
    def __init__(self, num_classes=2):
        super(AudioClassifier, self).__init__()
        
        # CNN layers
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout = nn.Dropout(0.3)
        self.relu = nn.ReLU()
        
        # LSTM layers
        self.lstm = nn.LSTM(
            input_size=128,  # Match features dimension
            hidden_size=256,  # Configurable
            num_layers=2,  # Configurable
            batch_first=True,  # Ensure batch is the first dimension
            bidirectional=True  # Enable if bidirectional LSTM is needed
        )
        
        # Fully connected layers
        self.fc1 = nn.Linear(256 * 2, 128)  # *2 because bidirectional LSTM (512 -> 128)
        self.fc2 = nn.Linear(128, num_classes)
        
    def forward(self, x):
        # print(f"Initial input shape: {x.shape}")  # [batch_size, 1, freq, time]

        batch_size, channels, freq, time = x.shape

        # Permute to [batch_size, time, channels, freq]
        x = x.permute(0, 3, 1, 2)
        # print(f"Shape after permute: {x.shape}")  # [batch_size, time, channels, freq]

        # Reshape to [batch_size, time, features]
        x = x.reshape(batch_size, time, -1)  # -1 computes channels * freq automatically
        # print(f"Shape after reshape: {x.shape}")  # [batch_size, time, features]

        # Pass through LSTM
        x, _ = self.lstm(x)  # [batch_size, time, hidden_size * (1 or 2)]
        # print(f"Shape after LSTM: {x.shape}")

        # Take the last time step
        x = x[:, -1, :]  # [batch_size, hidden_size * (1 or 2)]
        # print(f"Shape after selecting last time step: {x.shape}")

        # Fully connected layer
        x = self.fc1(x)  # [batch_size, 128]
        # print(f"Shape after fc1: {x.shape}")

        x = self.fc2(x)  # [batch_size, num_classes]
        # print(f"Shape after fc2: {x.shape}")

        return x



In [84]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = AudioClassifier(num_classes=2).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

for epoch in range(20):
    # Training
    model.train()
    train_loss = 0.0
    for spectrograms, labels in train_loader:
        spectrograms, labels = spectrograms.to(device), labels.to(device)
        outputs = model(spectrograms)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    
    print(f"Epoch {epoch + 1}, Training Loss: {train_loss / len(train_loader):.4f}")
    
    # Testing
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for spectrograms, labels in test_loader:
            spectrograms, labels = spectrograms.to(device), labels.to(device)
            outputs = model(spectrograms)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            
            # Calculate accuracy
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    print(f"Testing Loss: {test_loss / len(test_loader):.4f}, Accuracy: {100 * correct / total:.2f}%")

Epoch 1, Training Loss: 0.2866
Testing Loss: 0.4169, Accuracy: 87.28%
Epoch 2, Training Loss: 0.2318
Testing Loss: 0.2724, Accuracy: 89.47%
Epoch 3, Training Loss: 0.2190
Testing Loss: 0.2113, Accuracy: 91.03%
Epoch 4, Training Loss: 0.2099
Testing Loss: 0.2167, Accuracy: 91.26%
Epoch 5, Training Loss: 0.2093
Testing Loss: 0.2123, Accuracy: 91.43%
Epoch 6, Training Loss: 0.2048
Testing Loss: 0.2085, Accuracy: 91.43%
Epoch 7, Training Loss: 0.2095
Testing Loss: 0.2064, Accuracy: 91.26%
Epoch 8, Training Loss: 0.2034
Testing Loss: 0.1950, Accuracy: 92.11%
Epoch 9, Training Loss: 0.2050
Testing Loss: 0.2069, Accuracy: 91.16%
Epoch 10, Training Loss: 0.1978
Testing Loss: 0.1884, Accuracy: 92.21%
Epoch 11, Training Loss: 0.2085
Testing Loss: 0.3805, Accuracy: 75.69%
Epoch 12, Training Loss: 0.2244
Testing Loss: 0.2092, Accuracy: 91.66%
Epoch 13, Training Loss: 0.2057
Testing Loss: 0.1961, Accuracy: 91.99%
Epoch 14, Training Loss: 0.1927
Testing Loss: 0.1942, Accuracy: 92.19%
Epoch 15, Train

In [94]:
all_labels = []
all_predictions = []

model.eval()  # Set model to evaluation mode (no gradients)
correct = 0
total = 0

with torch.no_grad():
    for spectrograms, labels in test_loader:
        spectrograms, labels = spectrograms.to(device), labels.to(device)
        outputs = model(spectrograms)
        _, predicted = torch.max(outputs, 1)
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy: {100 * correct / total:.2f}%")
conf = confusion_matrix(all_labels, all_predictions)
print(conf)


Accuracy: 91.84%
[[2950   54]
 [ 270  695]]


In [85]:
# save model 
# torch.save(model.state_dict(), "audio_classifier_second.pth")
torch.save(model.state_dict(), "audio_classifier_third.pth")

In [72]:
# pass the audio segments through the model
predictions = []
for segment in tqdm(audio_segments):
    # extract spectrogram
    spectrogram = preprocess_audio([segment[0]])
    spectrogram = spectrogram.to(device)
    output = model(spectrogram)
    _, predicted = torch.max(output, 1)
    predictions.append(predicted.item())


  0%|          | 0/8060 [00:00<?, ?it/s]c:\Users\Jayesh\AppData\Local\Programs\Python\Python39\lib\site-packages\torchaudio\functional\functional.py:584: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  warnings.warn(
100%|██████████| 8060/8060 [08:43<00:00, 15.39it/s]


In [87]:
# count 0s and 1s 
pd.Series(predictions).value_counts()

0    6083
1    1977
Name: count, dtype: int64

In [92]:
# count original 0s and 1s
count = 0
for segment in audio_segments:
    if segment[1] == 1:
        count += 1

In [93]:
count

3861

In [90]:
# iterate through predictions where every 3 seconds is a prediction and there is a 1 second overlap so count accordingly
time_drinking = 0
current_time_drinking = 0
in_drinking_event = False
for i in predictions:
    if i == 1:    
        if not in_drinking_event:
            current_time_drinking += 2
            in_drinking_event = True
        elif in_drinking_event:
            time_drinking += 1
    else:
        in_drinking_event = False
        time_drinking += current_time_drinking
        # if current_time_drinking > 2:
        current_time_drinking = 0
        # else:
        #     current_time_drinking = 0

In [91]:
time_drinking

2507